## Import

Load data.

In [5]:
import os
import pandas as pd
import pinyin
from datetime import date
from opencc import OpenCC
from utils import *

save = True
delim = "|"
timestamp = date.today().strftime("%Y%m%d")
in_path = os.path.join("input", "pleco_20230529.csv")
out_path = os.path.join("output", f"pleco_{timestamp}.csv")
audio_dir = os.path.join("output", "audio")

cols = ["Simp", "Trad", "Meaning", "Simp (Color)", "Trad (Color)", "Pinyin (Color)", "Sound", "Silhouette"]
df = pd.read_csv(in_path, dtype = str).rename(columns = {"Pinyin": "Pinyin (Color)"})
df = df[[col for col in cols if col in df.columns]]
df_edited = df.copy()

print("Shape:", df.shape)
df.head()

Shape: (150, 4)


,Simp,Trad,Meaning,Pinyin (Color)
0,苍白,蒼白,adjective 1 pale; pallid; wan 2 lifeless; flat...,cāngbái
1,细瘦,細瘦,adjective thin; scrawny,xìshòu
2,裂缝,裂縫,noun 1 rift; crevice; crack; fissure 2 figurat...,lièfèng
3,仿佛,仿佛,adverb seemingly; as if verb be more or less ...,fǎngfú
4,黏性,黏性,noun stickiness; adhesiveness; viscidity; visc...,niánxìng


# Basic Formatting

In [7]:
df_edited["Silhouette"] = df["Simp"].apply(get_silhouette)
df_edited = df_edited[[col for col in cols if col in df_edited.columns]]

print("\nView table:")
df_edited


View table:


,Simp,Trad,Meaning,Pinyin (Color),Silhouette
0,苍白,蒼白,adjective 1 pale; pallid; wan 2 lifeless; flat...,cāngbái,_ _
1,细瘦,細瘦,adjective thin; scrawny,xìshòu,_ _
2,裂缝,裂縫,noun 1 rift; crevice; crack; fissure 2 figurat...,lièfèng,_ _
3,仿佛,仿佛,adverb seemingly; as if verb be more or less ...,fǎngfú,_ _
4,黏性,黏性,noun stickiness; adhesiveness; viscidity; visc...,niánxìng,_ _
...,...,...,...,...,...
145,遮蔽,遮蔽,verb 1 hide from view; cover; screen 2 obstruc...,zhēbì,_ _
146,乌鸦,烏鴉,noun crow,wūyā,_ _
147,塔楼,塔樓,noun 1 tower 2 turret 3 tower block (a residen...,tǎlóu,_ _
148,缭绕,繚繞,verb literary 1 curl up; wind around 2 entangle,liáorào,_ _


## Coloring

Convert characters to pinyin. Then add color.

In [4]:
pin_split = df_edited["Simp"].apply(pinyin.get, args = delim).apply(replace_special_char).apply(full_split, args = delim)
simp_split = df_edited["Simp"].apply(delim.join).apply(full_split, args = delim)
trad_split = df_edited["Trad"].apply(delim.join).apply(full_split, args = delim)

df_edited["Simp (Color)"] = pd.concat([pin_split, simp_split], axis = 1).apply(lambda row: add_color_hanzi(row[0], row[1]), axis = 1)
df_edited["Trad (Color)"] = pd.concat([pin_split, trad_split], axis = 1).apply(lambda row: add_color_hanzi(row[0], row[1]), axis = 1)
df_edited["Pinyin (Color)"] = df["Pinyin (Color)"].apply(pinyin.get, args = delim)

df_edited = df_edited[[col for col in cols if col in df_edited.columns]]
df_edited.head()

,Simp,Trad,Meaning,Simp (Color),Trad (Color),Pinyin (Color),Silhouette
0,苍白,蒼白,adjective 1 pale; pallid; wan 2 lifeless; flat...,<span class = 'tone1'>苍</span><span class = 't...,<span class = 'tone1'>蒼</span><span class = 't...,cāngbái,_ _
1,细瘦,細瘦,adjective thin; scrawny,<span class = 'tone4'>细瘦</span>,<span class = 'tone4'>細瘦</span>,xìshòu,_ _
2,裂缝,裂縫,noun 1 rift; crevice; crack; fissure 2 figurat...,<span class = 'tone4'>裂</span><span class = 't...,<span class = 'tone4'>裂</span><span class = 't...,lièfèng,_ _
3,仿佛,仿佛,adverb seemingly; as if verb be more or less ...,<span class = 'tone3'>仿</span><span class = 't...,<span class = 'tone3'>仿</span><span class = 't...,fǎngfú,_ _
4,黏性,黏性,noun stickiness; adhesiveness; viscidity; visc...,<span class = 'tone2'>黏</span><span class = 't...,<span class = 'tone2'>黏</span><span class = 't...,niánxìng,_ _


# Sound

In [4]:
df_edited["Sound"] = df_edited["Simp"].apply(lambda s : save_audio(s, audio_dir, save))

errors = df_edited["Sound"][df_edited["Sound"].apply(lambda s : s.startswith("FAILED"))]
print("Errors:\n", errors)
df_edited.head()

Errors:
 1653    FAILED WRITING: output\audio\调 (w/ t).mp3
Name: Sound, dtype: object


,Simp,Trad,Meaning,Simp (Color),Trad (Color),Pinyin (Color),Sound,Silhouette
0,滑稽,滑稽,funny (w/ h)<br>comical<br>amusing,<span class = 'tone2'>滑</span><span class = 't...,<span class = 'tone2'>滑</span><span class = 't...,<span class = 'tone2'>huá</span><span class = ...,output\audio\滑稽.mp3,_ _
1,不禁,不禁,"""can't help (doing something)""",<span class = 'tone4'>不禁</span>,<span class = 'tone4'>不禁</span>,<span class = 'tone4'>bùjìn</span>,output\audio\不禁.mp3,_ _
2,湖面,湖面,surface of a lake,<span class = 'tone2'>湖</span><span class = 't...,<span class = 'tone2'>湖</span><span class = 't...,<span class = 'tone2'>hú</span><span class = '...,output\audio\湖面.mp3,_ _
3,打翻,打翻,to knock over<br>to overturn<br>to overthrow,<span class = 'tone3'>打</span><span class = 't...,<span class = 'tone3'>打</span><span class = 't...,<span class = 'tone3'>dǎ</span><span class = '...,output\audio\打翻.mp3,_ _
4,友谊,友誼,friendship<br>companionship,<span class = 'tone3'>友</span><span class = 't...,<span class = 'tone3'>友</span><span class = 't...,<span class = 'tone3'>yǒu</span><span class = ...,output\audio\友谊.mp3,_ _


# Save

In [5]:
if save:
    df_edited.to_csv(out_path, index = False)